In [0]:
!nvidia-smi

Thu Aug 15 22:47:51 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5901, done.
remote: Total 5901 (delta 0), reused 0 (delta 0), pack-reused 5901
Receiving objects: 100% (5901/5901), 6.16 MiB | 31.54 MiB/s, done.
Resolving deltas: 100% (3925/3925), done.


In [0]:
cd darknet

/content/darknet


In [0]:
ls

cfg/       include/      LICENSE.gen   LICENSE.mit  python/    src/
data/      LICENSE       LICENSE.gpl   LICENSE.v1   README.md
examples/  LICENSE.fuck  LICENSE.meta  Makefile     scripts/


In [0]:
!sed -i 's/GPU=0/GPU=1/g' Makefile

In [0]:
!cat Makefile

GPU=1
CUDNN=0
OPENCV=0
OPENMP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52]
#      -gencode arch=compute_20,code=[sm_20,sm_21] \ This one is deprecated?

# This is what I use, uncomment if you know your arch and want to specify
# ARCH= -gencode arch=compute_52,code=compute_52

VPATH=./src/:./examples
SLIB=libdarknet.so
ALIB=libdarknet.a
EXEC=darknet
OBJDIR=./obj/

CC=gcc
CPP=g++
NVCC=nvcc 
AR=ar
ARFLAGS=rcs
OPTS=-Ofast
LDFLAGS= -lm -pthread 
COMMON= -Iinclude/ -Isrc/
CFLAGS=-Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC

ifeq ($(OPENMP), 1) 
CFLAGS+= -fopenmp
endif

ifeq ($(DEBUG), 1) 
OPTS=-O0 -g
endif

CFLAGS+=$(OPTS)

ifeq ($(OPENCV), 1) 
COMMON+= -DOPENCV
CFLAGS+= -DOPENCV
LDFLAGS+= `pkg-config --libs opencv` -lstdc++
COMMON+= `pkg-config --cflags opencv` 
endif

ifeq ($(GPU), 1) 
COMMON+= -DGPU -I/u

In [0]:
!pip install opencv-python

In [0]:
ls

cfg/       include/      LICENSE.gen   LICENSE.mit  python/    src/
data/      LICENSE       LICENSE.gpl   LICENSE.v1   README.md
examples/  LICENSE.fuck  LICENSE.meta  Makefile     scripts/


In [0]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:947:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o


In [0]:
ls

backup/   examples/       LICENSE       LICENSE.meta  obj/       scripts/
cfg/      include/        LICENSE.fuck  LICENSE.mit   python/    src/
darknet*  libdarknet.a    LICENSE.gen   LICENSE.v1    README.md
data/     libdarknet.so*  LICENSE.gpl   Makefile      results/


In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2019-08-15 22:54:08--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  74.1MB/s    in 3.2s    

2019-08-15 22:54:11 (74.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
ls

backup/   examples/       LICENSE       LICENSE.meta  obj/       scripts/
cfg/      include/        LICENSE.fuck  LICENSE.mit   python/    src/
darknet*  libdarknet.a    LICENSE.gen   LICENSE.v1    README.md  yolov3.weights
data/     libdarknet.so*  LICENSE.gpl   Makefile      results/


In [0]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

In [0]:
ls

backup/    include/        LICENSE.gen   Makefile         results/
cfg/       libdarknet.a    LICENSE.gpl   obj/             scripts/
darknet*   libdarknet.so*  LICENSE.meta  predictions.jpg  src/
data/      LICENSE         LICENSE.mit   python/          yolov3.weights
examples/  LICENSE.fuck    LICENSE.v1    README.md


In [0]:
pwd

'/content/darknet'

In [0]:
cd data

/content/darknet/data


In [0]:
ls

9k.labels   coco.names   goal.txt                  inet9k.map        People.zip
9k.names    dog.jpg      horses.jpg                kite.jpg          person.jpg
9k.tree     eagle.jpg    imagenet.labels.list      labels/           scream.jpg
coco9k.map  giraffe.jpg  imagenet.shortnames.list  openimages.names  voc.names


In [0]:
!unzip People.zip

Archive:  People.zip
   creating: People/
  inflating: People/action-1834465_960_720.jpg  
  inflating: People/actor-1807557_960_720.jpg  
  inflating: People/adult-1807500_960_720.jpg  
  inflating: People/adult-1822695_960_720.jpg  
  inflating: People/adult-2449725_960_720.jpg  
  inflating: People/adult-3086307_960_720.jpg  
  inflating: People/affection-1854081_960_720.jpg  
  inflating: People/agriculture-1807549_960_720.jpg  
  inflating: People/airport-2373727_960_720.jpg  
  inflating: People/airport-4120835_960_720.jpg  
  inflating: People/american-soldier-381847_960_720.jpg  
  inflating: People/analytics-2697949_960_720.jpg  
  inflating: People/analyzing-people-3441040_960_720.jpg  
  inflating: People/animals-1793409_960_720.jpg  
  inflating: People/apple-2712135_960_720.jpg  
  inflating: People/architect-1080589_960_720.jpg  
  inflating: People/asia-1793414_960_720.jpg  
  inflating: People/asia-1793425_960_720.jpg  
  inflating: People/asia-1822411_960_720.jpg  
  i

In [0]:
ls

9k.labels   dog.jpg      imagenet.labels.list      openimages.names  voc.names
9k.names    eagle.jpg    imagenet.shortnames.list  People/
9k.tree     giraffe.jpg  inet9k.map                People.zip
coco9k.map  goal.txt     kite.jpg                  person.jpg
coco.names  horses.jpg   labels/                   scream.jpg


In [0]:
cd ..

/content/darknet


In [0]:
ls

backup/    include/        LICENSE.gen   Makefile         results/
cfg/       libdarknet.a    LICENSE.gpl   obj/             scripts/
darknet*   libdarknet.so*  LICENSE.meta  predictions.jpg  src/
data/      LICENSE         LICENSE.mit   python/          yolov3.weights
examples/  LICENSE.fuck    LICENSE.v1    README.md


In [0]:
pwd

'/content/darknet'

In [0]:
!python3 data/People/process.py

In [0]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2019-08-15 22:59:49--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  19.4MB/s    in 8.0s    

2019-08-15 22:59:57 (19.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [0]:
!./darknet detector train cfg/obj.data cfg/obj.cfg darknet53.conv.74

obj
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 15

backup/            libdarknet.a    LICENSE.mit      results/
cfg/               libdarknet.so*  LICENSE.v1       scripts/
darknet*           LICENSE         Makefile         src/
darknet53.conv.74  LICENSE.fuck    obj/             test.txt
data/              LICENSE.gen     predictions.jpg  train.txt
examples/          LICENSE.gpl     python/          yolov3.weights
include/           LICENSE.meta    README.md
